In [1]:
# The following is from "big run"
import sys
sys.path.append('..')
from lib import *
import pandas as pd
import numpy as np

# Setup
lsa_np = np.load('../data/parsed/lsa_output.npy')
metadata = pd.read_pickle('../data/parsed/pickles/pickled_data_test.pickle')
metadata = metadata.loc[metadata['Scenario'] == '401']
metadata = metadata.reset_index(drop=True)
lsa_df = pd.DataFrame(lsa_np)
df = pd.concat([metadata, lsa_df], axis=1, join_axes=[metadata.index])
df = df.loc[df['Label'] != '-1']
df = df.reset_index(drop=True)
cat_features = ['To','From']
features = list(range(100))
features.extend(cat_features + ['Date'])
# features.extend(cat_features + ['ID'])

df = df[features + ['Label'] + ['ID']]
# df = df[features + ['ID']]

In [ ]:
# TODO: test tree behavior in ml_master when given .2 of training set to work with
# TODO: rather than making each tree its own process, divide the number of trees by the number of cores, and create
#       a list of trees that can be processed linearly in its own core.s

num_trees = 4
tries = 1
num_increments = 999
random_seed = 600
random_seed = None
overall_training_ratio = .8
initial_training_ratio = .2
increment_size = 50
cross_val_rnf_incremental(num_trees, df, tries, num_increments, random_seed,cat_features, overall_training_ratio,
                          initial_training_ratio, increment_size)

# Built-in incremental learning vs trees training on larger initial sizes

## Initializing control variables

In [2]:
n_trees = 64
tree_depth = 100
random_seed = 42
n_max_features = 11
cat_features = ['To', 'From']

## Forests Trained on increasing datasets

In [ ]:
forest_100 = RNF(df, n_trees, tree_depth, random_seed, n_max_features, 100, cat_features)
forest_200 = RNF(df, n_trees, tree_depth, random_seed, n_max_features, 200, cat_features)
forest_300 = RNF(df, n_trees, tree_depth, random_seed, n_max_features, 300, cat_features)
forest_400 = RNF(df, n_trees, tree_depth, random_seed, n_max_features, 400, cat_features)
forest_500 = RNF(df, n_trees, tree_depth, random_seed, n_max_features, 500, cat_features)
incremental_forests = [forest_100, forest_200, forest_300, forest_400, forest_500]

In [ ]:
for forest in incremental_forests:
    forest.fit_parallel()

In [ ]:
# evaluation
for forest in incremental_forests:
    print(evalStats(forest.predict_parallel(df[-100:])[1], df[-100:]), end='\n\n')

##  Incremental Forests

In [3]:
incremental_forest = RNF(df[0:100], n_trees, tree_depth, random_seed, n_max_features, 100, cat_features)

In [4]:
incremental_forest.fit_parallel()

In [5]:
print(evalStats(incremental_forest.predict_parallel(df[-100:])[1], df[-100:]))

(0.2, 1.0, 0.72, 0.33333333333333337)


In [6]:
incremental_forest.update(df[100:200].reset_index(drop=True))

=== test data ===
           0         1         2         3         4         5         6  \
0   0.057011 -0.062502 -0.046179  0.030211  0.035435 -0.045645 -0.029326   
4   0.108817 -0.108664 -0.027746 -0.009503  0.075909 -0.067194  0.002776   
9   0.074553 -0.119667 -0.058158 -0.046712 -0.076209  0.039655 -0.067659   
10  0.104754 -0.114982 -0.070301 -0.004543  0.092356 -0.091692 -0.040035   
11  0.100323 -0.140918 -0.101959 -0.056458 -0.020620  0.009814 -0.091434   
12  0.266829 -0.081587  0.042761 -0.003225 -0.025975 -0.032146  0.283848   
15  0.198445 -0.199616 -0.072364  0.010087  0.059218 -0.094201 -0.031724   
16  0.019109 -0.024481 -0.021611 -0.007056  0.022568 -0.023152 -0.013765   
18  0.192426 -0.192509 -0.009595 -0.022894 -0.056956 -0.004012  0.047629   
19  0.095630 -0.082462 -0.015825  0.010744  0.011367 -0.033602  0.003701   
22  0.110890 -0.108218 -0.039975  0.071851  0.063821 -0.013078  0.077183   
24  0.029554 -0.037538 -0.030473 -0.007673  0.026529 -0.029655 -0.0095

           0         1         2         3         4         5         6  \
1   0.034280 -0.043339 -0.001415  0.004682  0.030849 -0.026203 -0.031602   
2   0.065891 -0.086303 -0.029402  0.025286  0.061844 -0.060975 -0.053298   
3   0.114246 -0.139360 -0.064041 -0.021835  0.034806 -0.028799 -0.028732   
6   0.069938 -0.069297 -0.004973  0.004057  0.029771 -0.031491  0.017731   
10  0.104754 -0.114982 -0.070301 -0.004543  0.092356 -0.091692 -0.040035   
17  0.202837 -0.201797 -0.072976  0.031558  0.073843 -0.118034  0.026907   
19  0.095630 -0.082462 -0.015825  0.010744  0.011367 -0.033602  0.003701   
20  0.199201 -0.205835 -0.107358  0.018043  0.087584 -0.128294 -0.015824   
23  0.336686 -0.130565  0.020771 -0.042147 -0.141552  0.086784  0.068882   
26  0.042452 -0.042677 -0.034224 -0.007148  0.075606 -0.051492 -0.042591   
27  0.018296 -0.022782 -0.015482 -0.006475  0.010157 -0.019573  0.037778   
30  0.092305 -0.122690 -0.144505  0.139905 -0.007991 -0.057906 -0.026773   
34  0.142542

=== test data ===
           0         1         2         3         4         5         6  \
1   0.034280 -0.043339 -0.001415  0.004682  0.030849 -0.026203 -0.031602   
9   0.074553 -0.119667 -0.058158 -0.046712 -0.076209  0.039655 -0.067659   
10  0.104754 -0.114982 -0.070301 -0.004543  0.092356 -0.091692 -0.040035   
11  0.100323 -0.140918 -0.101959 -0.056458 -0.020620  0.009814 -0.091434   
21  0.141707 -0.125932  0.017564  0.000632 -0.039987  0.063162  0.070344   
23  0.336686 -0.130565  0.020771 -0.042147 -0.141552  0.086784  0.068882   
25  0.118229 -0.106633  0.038910 -0.013799 -0.070349  0.030047  0.114140   
27  0.018296 -0.022782 -0.015482 -0.006475  0.010157 -0.019573  0.037778   
29  0.075549 -0.068230 -0.050473 -0.011385  0.098911 -0.071011 -0.035143   
33  0.119135 -0.117287 -0.044649 -0.018929  0.034067 -0.049732  0.006066   
40  0.197338 -0.159787  0.001085 -0.028428 -0.051131  0.049003  0.075260   
42  0.238816 -0.099324  0.081894  0.019236  0.022499 -0.050243  0.2253

=== test data ===
           0         1         2         3         4         5         6  \
4   0.108817 -0.108664 -0.027746 -0.009503  0.075909 -0.067194  0.002776   
5   0.059497 -0.086650 -0.045897 -0.017851  0.003776 -0.013130 -0.039844   
10  0.104754 -0.114982 -0.070301 -0.004543  0.092356 -0.091692 -0.040035   
11  0.100323 -0.140918 -0.101959 -0.056458 -0.020620  0.009814 -0.091434   
13  0.137211 -0.180887 -0.102199 -0.013923  0.037832 -0.051548 -0.044314   
14  0.091810 -0.127273 -0.017061 -0.040226 -0.094257  0.047452 -0.022338   
20  0.199201 -0.205835 -0.107358  0.018043  0.087584 -0.128294 -0.015824   
22  0.110890 -0.108218 -0.039975  0.071851  0.063821 -0.013078  0.077183   
26  0.042452 -0.042677 -0.034224 -0.007148  0.075606 -0.051492 -0.042591   
30  0.092305 -0.122690 -0.144505  0.139905 -0.007991 -0.057906 -0.026773   
35  0.033921 -0.048418 -0.053163  0.046306  0.016123 -0.039386  0.019794   
36  0.139481 -0.157134 -0.099207  0.005114  0.082805 -0.107840 -0.0530

           0         1         2         3         4         5         6  \
0   0.057011 -0.062502 -0.046179  0.030211  0.035435 -0.045645 -0.029326   
1   0.034280 -0.043339 -0.001415  0.004682  0.030849 -0.026203 -0.031602   
3   0.114246 -0.139360 -0.064041 -0.021835  0.034806 -0.028799 -0.028732   
6   0.069938 -0.069297 -0.004973  0.004057  0.029771 -0.031491  0.017731   
7   0.068852 -0.072200 -0.070660  0.087205  0.203293  0.101565 -0.024429   
9   0.074553 -0.119667 -0.058158 -0.046712 -0.076209  0.039655 -0.067659   
11  0.100323 -0.140918 -0.101959 -0.056458 -0.020620  0.009814 -0.091434   
14  0.091810 -0.127273 -0.017061 -0.040226 -0.094257  0.047452 -0.022338   
15  0.198445 -0.199616 -0.072364  0.010087  0.059218 -0.094201 -0.031724   
20  0.199201 -0.205835 -0.107358  0.018043  0.087584 -0.128294 -0.015824   
21  0.141707 -0.125932  0.017564  0.000632 -0.039987  0.063162  0.070344   
22  0.110890 -0.108218 -0.039975  0.071851  0.063821 -0.013078  0.077183   
23  0.336686

...

[31 rows x 105 columns]
=== complement ===
{5, 7, 8, 11, 13, 14, 19, 33, 35, 36, 37, 41, 42, 48, 49, 50, 52, 57, 63, 66, 72, 74, 77, 78, 79, 80, 81, 84, 87, 89, 92}
=== test data ===
           0         1         2         3         4         5         6  \
3   0.114246 -0.139360 -0.064041 -0.021835  0.034806 -0.028799 -0.028732   
7   0.068852 -0.072200 -0.070660  0.087205  0.203293  0.101565 -0.024429   
8   0.080798 -0.106565 -0.067723 -0.010440  0.011666 -0.014577 -0.041043   
12  0.266829 -0.081587  0.042761 -0.003225 -0.025975 -0.032146  0.283848   
15  0.198445 -0.199616 -0.072364  0.010087  0.059218 -0.094201 -0.031724   
17  0.202837 -0.201797 -0.072976  0.031558  0.073843 -0.118034  0.026907   
20  0.199201 -0.205835 -0.107358  0.018043  0.087584 -0.128294 -0.015824   
21  0.141707 -0.125932  0.017564  0.000632 -0.039987  0.063162  0.070344   
27  0.018296 -0.022782 -0.015482 -0.006475  0.010157 -0.019573  0.037778   
30  0.092305 -0.122690 -0.144505  0.139905 -0.007991 -0.0

=== test data ===
           0         1         2         3         4         5         6  \
1   0.034280 -0.043339 -0.001415  0.004682  0.030849 -0.026203 -0.031602   
4   0.108817 -0.108664 -0.027746 -0.009503  0.075909 -0.067194  0.002776   
7   0.068852 -0.072200 -0.070660  0.087205  0.203293  0.101565 -0.024429   
8   0.080798 -0.106565 -0.067723 -0.010440  0.011666 -0.014577 -0.041043   
13  0.137211 -0.180887 -0.102199 -0.013923  0.037832 -0.051548 -0.044314   
15  0.198445 -0.199616 -0.072364  0.010087  0.059218 -0.094201 -0.031724   
18  0.192426 -0.192509 -0.009595 -0.022894 -0.056956 -0.004012  0.047629   
19  0.095630 -0.082462 -0.015825  0.010744  0.011367 -0.033602  0.003701   
21  0.141707 -0.125932  0.017564  0.000632 -0.039987  0.063162  0.070344   
22  0.110890 -0.108218 -0.039975  0.071851  0.063821 -0.013078  0.077183   
26  0.042452 -0.042677 -0.034224 -0.007148  0.075606 -0.051492 -0.042591   
27  0.018296 -0.022782 -0.015482 -0.006475  0.010157 -0.019573  0.0377

[42 rows x 105 columns]
=== complement ===
{1, 4, 7, 8, 13, 15, 18, 19, 21, 22, 26, 27, 29, 35, 37, 38, 39, 40, 42, 43, 44, 45, 46, 48, 50, 54, 56, 66, 67, 72, 74, 75, 77, 84, 86, 90, 91, 93, 94, 96, 97, 99}
=== test data ===
           0         1         2         3         4         5         6  \
0   0.057011 -0.062502 -0.046179  0.030211  0.035435 -0.045645 -0.029326   
1   0.034280 -0.043339 -0.001415  0.004682  0.030849 -0.026203 -0.031602   
2   0.065891 -0.086303 -0.029402  0.025286  0.061844 -0.060975 -0.053298   
4   0.108817 -0.108664 -0.027746 -0.009503  0.075909 -0.067194  0.002776   
7   0.068852 -0.072200 -0.070660  0.087205  0.203293  0.101565 -0.024429   
8   0.080798 -0.106565 -0.067723 -0.010440  0.011666 -0.014577 -0.041043   
9   0.074553 -0.119667 -0.058158 -0.046712 -0.076209  0.039655 -0.067659   
12  0.266829 -0.081587  0.042761 -0.003225 -0.025975 -0.032146  0.283848   
14  0.091810 -0.127273 -0.017061 -0.040226 -0.094257  0.047452 -0.022338   
19  0.095630 -

=== test data ===
           0         1         2         3         4         5         6  \
4   0.108817 -0.108664 -0.027746 -0.009503  0.075909 -0.067194  0.002776   
6   0.069938 -0.069297 -0.004973  0.004057  0.029771 -0.031491  0.017731   
8   0.080798 -0.106565 -0.067723 -0.010440  0.011666 -0.014577 -0.041043   
13  0.137211 -0.180887 -0.102199 -0.013923  0.037832 -0.051548 -0.044314   
16  0.019109 -0.024481 -0.021611 -0.007056  0.022568 -0.023152 -0.013765   
29  0.075549 -0.068230 -0.050473 -0.011385  0.098911 -0.071011 -0.035143   
30  0.092305 -0.122690 -0.144505  0.139905 -0.007991 -0.057906 -0.026773   
31  0.025930 -0.034716 -0.020104 -0.005341  0.018772 -0.009442  0.025116   
35  0.033921 -0.048418 -0.053163  0.046306  0.016123 -0.039386  0.019794   
38  0.209268 -0.144863 -0.000908  0.049288  0.067349 -0.100862  0.193722   
39  0.121978 -0.105488 -0.028644 -0.000241  0.047057 -0.070334  0.044838   
40  0.197338 -0.159787  0.001085 -0.028428 -0.051131  0.049003  0.0752

became right child


In [7]:
print(evalStats(incremental_forest.predict_parallel(df[-100:])[1], df[-100:]))

(0.2, 0.875, 0.71, 0.32558139534883723)


In [ ]:
flattened_nodes = []
for tree in incremental_forest.trees:
    print('a')
    flattened_nodes.append(tree.traverse())

In [ ]:
for f in flattened_nodes:
    print(f)
    for node in f:
        if len(node.rows) == 0:
            print("oh no")